In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=9123)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13645 ( 0.0%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

In [10]:
import numpy.ma as ma
import numpy as np

In [33]:
a = np.array([[1,2,5,6], [4,5,7,8]])
mana = a.mean()*1.4
maska = a<mana

b = np.array([[6,2,7,1], [4,5,7,8]])
manb = b.mean()*1.4
maskb = b<manb

mask_tot = maska + maskb

mask_bma = b*(np.logical_not(mask_tot)) > a*(np.logical_not(mask_tot)) 
mask_amb = b*(np.logical_not(mask_tot)) < a*(np.logical_not(mask_tot)) 

c = a*mask_tot*0.1 + b*mask_tot*0.9 + mask_bma*b*0.9 + mask_amb*a*0.1
cm = a*0.1 + b*0.9

print(c, cm)

[5.5 2.  6.3 1.5] [5.5 2.  6.8 1.5]


In [36]:
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender

In [44]:
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from numpy import linalg as LA
import numpy as np
import numpy.ma as ma
import scipy.sparse as sps

In [66]:
class ItemKNNScoresHybridTwoRecommender2(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridTwoRecommender"


    def __init__(self, URM_train, Recommender_1, Recommender_2, verbose = True):
        super(ItemKNNScoresHybridTwoRecommender2, self).__init__(URM_train, verbose = verbose)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2
        
        
        
    def fit(self, alpha = 0.5):

        self.alpha = alpha

        

        '''
        print(f"CURRENT CONFIGURATION:\n{self.Recommender_1.RECOMMENDER_NAME} with weight alpha: {self.alpha}")
        print(f"{self.Recommender_2.RECOMMENDER_NAME} with weight beta: {1 - self.alpha}")
        '''

    def _compute_item_score(self, user_id_array = None, items_to_compute = None):
        
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)

        l2_1 = LA.norm(item_weights_1, 2)
        l2_1_scores = item_weights_1 / l2_1

        l2_2 = LA.norm(item_weights_2, 2)
        l2_2_scores = item_weights_2 / l2_2 
        
        item_weights = np.zeros_like(item_weights_1)
        
        print("AT USER:", user_id_array[0]
            
        for user in user_id_array:
            usr = user % len(user_id_array)
            mean_score_1 = l2_1_scores[usr, :].mean()
            mean_score_2 = l2_2_scores[usr, :].mean()
            mask1 = l2_1_scores[usr, :] < mean_score_1*1.4
            mask2 = l2_2_scores[usr, :] < mean_score_2*1.4
            
            mask_tot = mask1+mask2
            
            mask_neg = np.logical_not(mask_tot)
            mask_1m2 = l2_2_scores[usr, :]*mask_neg < l2_1_scores[usr, :]*mask_neg
            mask_2m1 = l2_2_scores[usr, :]*mask_neg > l2_1_scores[usr, :]*mask_neg
            
            item_weights[usr, :] = l2_1_scores[usr, :]*mask_tot*self.alpha + l2_2_scores[usr, :]*mask_tot*(1-self.alpha) + l2_1_scores[usr, :]*mask_1m2*self.alpha + l2_2_scores[usr, :]*mask_2m1*(1-self.alpha)

        #item_weights = l2_1_scores*self.alpha + l2_2_scores*(1 - self.alpha)

        return item_weights

In [54]:
recommender1 = SLIM_BPR_Cython(URM_train)
recommender2 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

model_init(recommender1, 'SLIMBPR', models_to_combine_best['SLIMBPR'])
model_init(recommender2, 'SLIMER', models_to_combine_best['SLIMER'])

../models_temp/Similarity_Hybrid/SLIMBPR.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMBPR'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/SLIMER.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMER'
SLIMElasticNetRecommender: Loading complete


In [40]:
def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNScoresHybridTwoRecommender2(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage_sco(recommender1, recommender2, evaluator_validation, 0.4, 0.6, 3)

AT USER: 0
AT USER: 1000
AT USER: 2001
AT USER: 3001
AT USER: 4002
AT USER: 5002
AT USER: 6003
AT USER: 7003
AT USER: 8003
AT USER: 9003
EvaluatorHoldout: Processed 10000 (73.3%) in 5.20 min. Users per second: 32
AT USER: 10003
AT USER: 11003
AT USER: 12004
AT USER: 13004
EvaluatorHoldout: Processed 13645 (100.0%) in 6.86 min. Users per second: 33
             MAP
cutoff          
10      0.247786
0.4 : 0.24778560086548354
AT USER: 0
AT USER: 1000
AT USER: 2001
AT USER: 3001
AT USER: 4002
AT USER: 5002
